<a href="https://colab.research.google.com/github/Premmikan/Comsys/blob/main/Premmika_NIEMPREM_ch02_alu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arithmetic Logic Unit (ALU)

*ดัดแปลงจากโน้ตบุ๊คต้นฉบับโดย ผศ.ดร.จิตร์ทัศน์ ฝักเจริญผล ภาคต้น ปีการศึกษา 2563 และอ้างอิงเนื้อหาจากตำรา [The Elements of Computing Systems โดย Nisan และ Schocken](https://www.nand2tetris.org/book)*

บทนี้เราจะสร้าง ALU ขึ้นจากเกทพื้นฐานต่าง ๆ ที่เราสร้างมาในบทที่แล้ว โดยอิงตาม ALU ของสถาปัตยกรรม Hack (สถาปัตยกรรมคอมพิวเตอร์ที่ใช้ในตำราเรียน) ซึ่งมีคุณสมบัติดังนี้
* ประมวลผลจำนวนเต็ม 16 บิตเท่านั้น ไม่รองรับ floating-point
* ใช้ระบบ Two's complement ในการแทนค่าติดลบ
* การคำนวณทางคณิตศาสตร์ที่รองรับคือ บวก ลบ กลับเครื่องหมาย เพิ่มค่าและลดค่าทีละหนึ่ง (`x+y`, `x-y`, `-x`, `x+1`, `x-1`)
* การคำนวณทางตรรกศาสตร์ที่รองรับคือ and or not (`x&y`, `x|y`, `!x`)
* ไม่รองรับการคูณและหาร (ยกภาระให้ซอฟต์แวร์)

เริ่มต้นด้วยการโหลดไลบรารีกันก่อน

In [ ]:
%%capture
!rm -rf comp-sys-public-lib
!wget -q -O - https://ecourse.cpe.ku.ac.th/courses/comsys/lib/ch02.tgz | tar zxf -
!pip install -e comp-sys-public-lib
!pip install -e comp-sys-public-lib/modules/component-builder
import site; site.main()
from course_ch02_init import *

ไลบรารีครั้งนี้เตรียมเกทพื้นฐานทั้งหมดที่สร้างขึ้นมาจากปฏิบัติการครั้งก่อนให้พร้อมใช้งานได้ทันทีโดยนิสิตไม่ต้องนิยามเอง ซึ่งได้แก่
[`Nand`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/nand.html)
[`Not`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/not.html)
[`And`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/and.html)
[`Or`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or.html)
[`Xor`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/xor.html)
[`Mux`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux.html)
[`DMux`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux.html)
[`Not16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/not16.html)
[`And16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/and16.html)
[`Or16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or16.html)
[`Mux16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux16.html)
[`Or8Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or8way.html)
[`Mux4Way16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux4way16.html)
[`Mux8Way16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux8way16.html)
[`DMux4Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux4way.html)
และ
[`DMux8Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux8way.html)

นอกเหนือจากเกทข้างต้น ไลบรารีสำหรับบทนี้ได้เตรียมอุปกรณ์ [`Buffer`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/buffer.html) เอาไว้ให้สำหรับเชื่อมต่อสายสัญญาณสองเส้นเข้าด้วยกัน ขาอินพุทและเอาท์พุทของ `Buffer` มีชื่อว่า `In` และ `out` เช่นเดียวกับ `Not` gate

In [ ]:
interact(Buffer)

# วงจรบวก 1 บิต: `HalfAdder` กับ `FullAdder`

เราจะเริ่มโดยการสร้างวงจรบวกขนาด 1 บิต ที่เป็นพื้นฐานในการสร้างวงจรคำนวณ  `HalfAdder` จะบวกอินพุทสองบิตเข้าด้วยกัน ในขณะที่ `FullAdder` จะรับอินพุท `c` เพิ่มเข้ามาอีกหนึ่งบิต ซึ่งมักจะแทนตัวทดจากผลลัพธ์การบวกของบิตในหลักนัยสำคัญต่ำกว่า โดยทั่วไปเราจะสร้าง `FullAdder` ขึ้นจาก `HalfAdder` สองตัว

ตัวอย่างการทำงานแบบโต้ตอบ
* [HalfAdder](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/halfadder.html)
* [FullAdder](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/fulladder.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class HalfAdder(Component):
    IN = [w.a, w.b]
    OUT = [w.sum, w.carry]

    PARTS = [
        Xor(a = w.a , b = w.b , out = w.sum),
        And(a = w.a, b = w.b, out = w.carry)
    ]

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class FullAdder(Component):
    IN = [w.a, w.b, w.c]
    OUT = [w.sum, w.carry]

    PARTS = [
        HalfAdder(a = w.a, b = w.b, sum = w.o1, carry = w.o2),
        HalfAdder(a = w.o1 , b = w.c, sum = w.sum, carry = w.o3),
        Or(a = w.o3, b = w.o2, out = w.carry)
    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(HalfAdder, depth=1)
interact(FullAdder, depth=1)

## Test cases

In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestHalfAdder(unittest.TestCase):
    def setUp(self):
        self.half_adder = HalfAdder()

    def test_00(self):
        self.assertEqual(self.half_adder.eval(a=_0, b=_0), {'sum':_0, 'carry':_0})

    def test_01(self):
        self.assertEqual(self.half_adder.eval(a=_0, b=_1), {'sum':_1, 'carry':_0})

    def test_10(self):
        self.assertEqual(self.half_adder.eval(a=_1, b=_0), {'sum':_1, 'carry':_0})

    def test_11(self):
        self.assertEqual(self.half_adder.eval(a=_1, b=_1), {'sum':_0, 'carry':_1})

run_test(TestHalfAdder)

....
----------------------------------------------------------------------
Ran 4 tests in 0.023s

OK


In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestFullAdder(unittest.TestCase):

    def setUp(self):
        self.full_adder = FullAdder()

    def test_00carry0(self):
        self.assertEqual(self.full_adder.eval(a=_0, b=_0, c=_0), {'sum':_0, 'carry':_0})

    def test_01carry0(self):
        self.assertEqual(self.full_adder.eval(a=_0, b=_1, c=_0), {'sum':_1, 'carry':_0})

    def test_10carry0(self):
        self.assertEqual(self.full_adder.eval(a=_1, b=_0, c=_0), {'sum':_1, 'carry':_0})

    def test_11carry0(self):
        self.assertEqual(self.full_adder.eval(a=_1, b=_1, c=_0), {'sum':_0, 'carry':_1})

    def test_00carry1(self):
        self.assertEqual(self.full_adder.eval(a=_0, b=_0, c=_1), {'sum':_1, 'carry':_0})

    def test_01carry1(self):
        self.assertEqual(self.full_adder.eval(a=_0, b=_1, c=_1), {'sum':_0, 'carry':_1})

    def test_10carry1(self):
        self.assertEqual(self.full_adder.eval(a=_1, b=_0, c=_1), {'sum':_0, 'carry':_1})

    def test_11carry1(self):
        self.assertEqual(self.full_adder.eval(a=_1, b=_1, c=_1), {'sum':_1, 'carry':_1})

run_test(TestFullAdder)

........
----------------------------------------------------------------------
Ran 8 tests in 0.050s

OK


# วงจรบวก 16 บิต: `Add16`, `Inc16`

จาก FullAdder ที่เราสร้างมา เราสามารถสร้างตัวบวก (adder) และตัวเพิ่มค่า (incrementer) ขนาด 16 บิต อุปกรณ์ที่สร้างตอนนี้จะไม่มีการตรวจสอบการล้นของค่าผลลัพธ์ (overflow)

ในการทำอุปกรณ์ทั้งสองอาจมีความจำเป็นต้องใช้ค่าคงที่ที่เคยแนะนำไว้ในแบบฝึกหัดที่แล้ว

* `w(width).T` แทนสัญญาณความกว้าง `width` ที่เป็น 1 ในทุกบิต
* `w(width).F` แทนสัญญาณความกว้าง `width` ที่เป็น 0 ในทุกบิต
* `w(width).constant(x)` แทนสัญญาณความกว้าง `width` ที่ให้ค่าเท่ากับ `x` เมื่อตีความเป็นเลขฐานสอง

ตัวอย่างการทำงานแบบโต้ตอบ
* [Add16](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/add16.html)
* [Inc16](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/inc16.html)

*คำเตือน:* บางคนอาจต้องการใช้ for loop ในการนิยามรายการ `FullAdder` ภายในอุปกรณ์ โดยประกาศสัญญาณสำหรับตัวทด (carry) เป็นแบบหลายบิตเช่น `w(16).c` แต่การทำเช่นนี้จะทำให้เกิดลูปขึ้นในวงจร เช่น เราอาจระบุให้ `FullAdder` ตัวหนึ่งรับอินพุทมาจาก `w.c[0]` และส่งเอาท์พุทไปที่ `w.c[1]` แม้ในความเป็นจริงจะเป็นสิ่งที่ทำได้ แต่ด้วยข้อจำกัดของไลบรารีจะทำให้มีการตีความว่าสัญญาณ​ `w.c` วนจากเอาท์พุทของ `FullAdder` กลับเข้ามาที่อินพุทของตนเอง ซึ่งไม่อนุญาตให้ทำ จึงแนะนำให้ตั้งชื่อสัญญาณลักษณะนี้แบบแยกตามบิตตรง ๆ เช่น `w.c0` `w.c1` ... หรือใช้ฟังก์ชัน [`getattr()`](https://docs.python.org/3/library/functions.html#getattr) ของไพทอนในการตั้งชื่อสายสัญญาณด้วยนิพจน์แบบสตริง

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class Add16(Component):
    IN = [w(16).a, w(16).b]
    OUT= [w(16).out]

    PARTS = [
        HalfAdder(a = w.a[0], b = w.b[0], sum = w.out[0], carry = w.c0)
    ]
    for i in range(1,16):
      PARTS.append(FullAdder(a = w.a[i], b = w.b[i], c = getattr(w,f"c{i-1}"), sum = w.out[i], carry = getattr(w,f"c{i}")))


In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class Inc16(Component):
    IN = [w(16).In]
    OUT= [w(16).out]

    PARTS = [
        Add16(a = w.In , b = w(16).constant(1), out = w.out)
    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(Add16, depth=0)
interact(Inc16, depth=1)

## Test cases
ให้แน่ใจว่ารันผ่านทุกเทสเคสโดยไม่มีข้อผิดพลาด

In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestAdd16(unittest.TestCase):
    def setUp(self):
        self.add16 = Add16()

    def test_0(self):
        self.assertEqual(self.add16.eval_single(a=Signal(0,16), b=Signal(0,16)), Signal(0,16))

    def test_1(self):
        self.assertEqual(self.add16.eval_single(a=Signal(0,16), b=Signal(1,16)), Signal(1,16))

    def test_2(self):
        self.assertEqual(self.add16.eval_single(a=Signal(1,16), b=Signal(1,16)), Signal(2,16))

    def test_5(self):
        self.assertEqual(self.add16.eval_single(a=Signal(2,16), b=Signal(3,16)), Signal(5,16))

    def test_128(self):
        self.assertEqual(self.add16.eval_single(a=Signal(32,16), b=Signal(96,16)), Signal(128,16))

    def test_1024(self):
        self.assertEqual(self.add16.eval_single(a=Signal(768,16), b=Signal(256,16)), Signal(1024,16))

run_test(TestAdd16)

......
----------------------------------------------------------------------
Ran 6 tests in 0.944s

OK


In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestInc16(unittest.TestCase):
    def setUp(self):
        self.inc16 = Inc16()

    def test_0(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(0,16)), Signal(1,16))

    def test_1(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(1,16)), Signal(2,16))

    def test_minus1(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(0b1111111111111111,16)), Signal(0,16))

    def test_5(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(5,16)), Signal(6,16))

    def test_minus5(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(0b1111111111111011,16)), Signal(0b1111111111111100,16))

    def test_768(self):
        self.assertEqual(self.inc16.eval_single(In=Signal(768,16)), Signal(769,16))

run_test(TestInc16)

......
----------------------------------------------------------------------
Ran 6 tests in 0.929s

OK


# การแทนจำนวนติดลบและการหาผลลบ

ในระบบ [Two's complement](https://en.wikipedia.org/wiki/Two%27s_complement) เราแทนจำนวนติดลบด้วยตัวเลข $n$ บิตดังนี้

$$ -x = \left\{\begin{array}{ll}
2^n - x & \mbox{if } x \neq 0 \\
0       & \mbox{otherwise}
\end{array}\right.$$

(ผู้ที่ผ่านรายวิชา Discrete Mathematics มาแล้วควรพึงระลึกได้ว่านิยามข้างต้นคือการมองจำนวนติดลบในระบบ modulo $2^n$)

ข้อดีของระบบนี้คือเราสามารถนำจำนวนมาบวกกันได้โดยตรงโดยไม่ต้องแยกแยะเงื่อนไขว่าจำนวนดังกล่าวเป็นค่าบวกหรือค่าลบ ดังนั้นการหาผลลบ $x-y$ จึงทำได้ด้วยการคำนวณหา $x + (-y)$ โดยตรงด้วยวงจรบวกที่สร้างเอาไว้แล้ว

## การคำนวณค่า $x-y$
ในมุมมองของฮาร์ดแวร์ เราสามารถสร้างสัญญาณแทนค่าของ $-x$ ได้โดยการกลับบิตของ $x$ และบวกด้วย 1 ทั้งนี้เนื่องจาก $2^n = (2^n - 1) + 1$ และ $2^n-1$ เมื่อเขียนในรูปไบนารีจะเท่ากับ 1 เขียนติดกัน $n-1$ ตัว (เช่น $2^3-1=7=111_2$ หรือ $2^8-1=255=11111111_2$) ค่าของ $111\ldots1_2-x$ จึงเทียบเท่ากับการกลับค่าบิตของ $x$ หรือเท่ากับ $!x$

ดังนั้นในระบบ Two's complement ค่าของ $-x$ และ $!x$ จึงมีความสัมพันธ์เป็น
$$-x = ~!x + 1$$

จากความสัมพันธ์ข้างต้น ประกอบกับการออกแบบอันแยบยลของสถาปัตยกรรม Hack (โดยผู้เขียนตำรา) เราสามารถคำนวณค่า $x-y$ ในระบบ Two's complement โดยใช้เพียงวงจรบวกและ Not gate ได้ดังนี้
$$\begin{eqnarray*}
x - y
&=& -(y-x) \\
&=& -(y~+~!x+1) \\
&=& -(y~+~!x)-1 \\
&=& !(y~+~!x)
\end{eqnarray*}$$

# การคำนวณทางตรรกศาสตร์

### การคำนวณ $!x$
เป็นการกลับบิตทั้งหมดของ $x$ ทำได้โดยใช้วงจร `Not16` ที่ออกแบบมาก่อนหน้านี้

### การคำนวณ $x \& y$
เป็นการนำบิตของ $x$ มา and กับบิตของ $y$ ทำได้โดยใช้วงจร `And16` ที่ออกแบบมาก่อนหน้านี้

### การคำนวณ $x | y$
เป็นการนำบิตของ $x$ มา or กับบิตของ $y$ แม้จะทำได้โดยใช้วงจร `Or16` แต่เราสามารถใช้[กฎของ De Morgan](https://en.wikipedia.org/wiki/De_Morgan%27s_laws) เพื่อคำนวณ $x|y$ ได้จากความสัมพันธ์

$$x|y = ~!(!x ~\&~ !y)$$

ซึ่งใช้เพียงเกท `Not` และ `And` ที่เราจำเป็นต้องใช้ในการลบเลขอยู่แล้ว

# การออกแบบ ALU
ตำราเรียนได้แนะนำแนวทางการออกแบบ ALU ที่ใช้ฮาร์ดแวร์อย่างประหยัด แต่สามารถคำนวณค่าได้หลากหลาย คุณลักษณะภายนอกและโครงสร้างภายในของ ALU เป็นดังนี้

![data-mem.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/alu-design.png)

`x` และ `y` เป็นอินพุทจำนวนเต็ม 16 บิต ส่วน `out` เป็นผลลัพธ์ที่คำนวณได้ ผลการคำนวณเป็นอย่างไรขึ้นอยู่กับค่าของบิตควบคุมดังนี้

* `zx` หากเป็น 1 จะเซ็ตค่า x' = 0 ก่อนส่งไปยังวงจรถัดไป
* `zy` หากเป็น 1 จะเซ็ตค่า y' = 0 ก่อนส่งไปยังวงจรถัดไป
* `nx` หากเป็น 1 จะเซ็ตค่า x'' = !x' ก่อนส่งไปยังวงจรถัดไป
* `ny` หากเป็น 1 จะเซ็ตค่า y'' = !y' ก่อนส่งไปยังวงจรถัดไป
* `f` หากเป็น 0 จะคำนวณ out' = x''&y'' หากเป็น 1 จะคำนวณ out' = x''+y''
* `no` หากเป็น 1 จะเซ็ตค่า out = !out'

ส่วนบิตสถานะเอาท์พุท `zr` และ `ng` มีความหมายดังนี้
* `zr` เป็น 1 เมื่อและต่อเมื่อ out = 0
* `ng` เป็น 1 เมื่อและต่อเมื่อบิตซ้ายสุด (นัยสำคัญสูงสุด) ของ out เป็น 1 (ผลลัพธ์เป็นค่าลบตามการตีความแบบ Two's complement)


ตารางด้านล่างสรุปพฤติกรรมการคำนวณของ ALU เมื่อกำหนดค่าของบิตควบคุมที่แตกต่างกัน

![data-mem.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/alu-table.png)

# ALU Part 1: ALU แบบที่ไม่มีการรายงานสถานะ

เราจะแบ่งการพัฒนา ALU ออกเป็นสองขั้นตอน กล่าวคือ ในขั้นแรก เราจะสร้าง ALU ที่ไม่มีการคืนสถานะ (`zr` กับ `ng`) ก่อน ตามที่มีการแนะนำไว้ในหนังสือ  สัญญาณทั้งสองจะถูกเพิ่มเข้ามาในขั้นถัดไป

ตัวอย่างการทำงานแบบโต้ตอบ
* [ALUwoStatus](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/aluwostatus.html)


In [ ]:
# DO NOT ERASE - to be graded
class ALUwoStatus(Component):
    IN = [w(16).x, w(16).y,
          w.zx, w.nx,
          w.zy, w.ny,
          w.f,
          w.no]

    OUT = [w(16).out]

    PARTS = [
        Mux16(a = w.x, b = w(16).constant(0), sel = w.zx, out = w(16).zerox),
        Mux16(a = w.y, b = w(16).constant(0), sel = w.zy , out = w(16).zeroy),

        Not16(In = w.zerox, out = w(16).notx),
        Mux16(a = w.zerox,b = w.notx, sel = w.nx, out = w(16).negx),

        Not16(In = w.zeroy, out = w(16).noty),
        Mux16(a = w.zeroy,b = w.noty, sel = w.ny, out = w(16).negy),

        Add16(a = w.negx,b = w.negy, out= w(16).o1),
        And16(a = w.negx,b = w.negy, out = w(16).o2),
        Mux16(a = w.o2, b = w.o1, sel = w.f, out = w(16).o3),

        Not16(In = w.o3, out = w(16).noto),
        Mux16(a = w.o3, b = w.noto, sel = w.no , out = w.out)
    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(ALUwoStatus, depth=1)

## Test cases

In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestALUwoStatus(unittest.TestCase):
    def setUp(self):
        self.alu = ALUwoStatus()

# x = 0, y = 1
    def test_0(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_0, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000000,16)})

    def test_1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16)})

    def test_Minus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16)})

    def test_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_0),
                         {'out':Signal(0b0000000000000000,16)})

    def test_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b1111111111111111,16)})

    def test_not_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111111111,16)})

    def test_not_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_1),
                         {'out':Signal(0b0000000000000000,16)})

    def test_minus_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000000,16)})

    def test_minus_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16)})

    def test_xplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16)})

    def test_yplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000000,16)})

    def test_xminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16)})

    def test_yminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111110,16)})

    def test_xplusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16)})

    def test_xminusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16)})

    def test_yminusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111111111,16)})

    def test_xandy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000000,16)})

    def test_xory(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_0, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111111111,16)})

# x = 17, y = 3
    def test2_0(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_0, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000000,16)})

    def test2_1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16)})

    def test2_minus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16)})

    def test2_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_0),
                         {'out':Signal(0b0000000000010001,16)})

    def test2_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000011,16)})

    def test2_not_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111101110,16)})

    def test2_not_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_1),
                         {'out':Signal(0b1111111111111100,16)})

    def test2_minusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111101111,16)})

    def test2_minusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b1111111111111101,16)})

    def test2_xplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000010010,16)})

    def test2_yplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000100,16)})

    def test2_xminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_0),
                         {'out':Signal(0b0000000000010000,16)})

    def test2_yminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000010,16)})

    def test2_xplusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000010100,16)})

    def test2_xminusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000001110,16)})

    def test2_yminusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111110010,16)})

    def test2_xandy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000001,16)})

    def test2_xory(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_0, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b0000000000010011,16)})

run_test(TestALUwoStatus)

....................................
----------------------------------------------------------------------
Ran 36 tests in 21.115s

OK


# ALU (part 2): ALU เต็มรูปแบบ

ในส่วนนี้ เราจะสร้าง ALU เต็มรูปแบบขึ้นจาก `ALUwoStatus` ที่นำเอาผลลัพธ์มาคำนวณบิตสถานะ `zr` และ `ng` เข้าไปด้วย

ตัวอย่างการทำงานแบบโต้ตอบ
* [ALU](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/alu.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

class ALU(Component):
    IN = [w(16).x, w(16).y,
          w.zx, w.nx,
          w.zy, w.ny,
          w.f,
          w.no]

    OUT = [w(16).out, w.zr, w.ng]

    PARTS = [
        ALUwoStatus(x = w(16).x,y = w(16).y,zx = w.zx,nx = w.nx,zy = w.zy,ny = w.ny,f = w.f,no = w.no , out = w(16).out),
        Or8Way(In = w.out[0:8], out = w.out1),
        Or8Way(In = w.out[8:16], out = w.out2),
        Or(a = w.out1,b = w.out2, out = w.out3),

        Not(In = w.out3, out = w.zr),

        Mux(a = w.F,b = w.T,sel = w.out[15], out = w.ng),

    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(ALU, depth=1)

## Test cases

In [ ]:
_0 = Signal(0)
_1 = Signal(1)

class TestALU(unittest.TestCase):
    def setUp(self):
        self.alu = ALU()

# x = 0, y = 1
    def test_0(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_0, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test_Minus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_0),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_not_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_not_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_1),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_minus_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_minus_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test_xplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test_yplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_xminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_yminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111110,16), 'zr':_0, 'ng':_1})

    def test_xplusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_xminusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test_yminusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test_xandy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test_xory(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000000000,16), y=Signal(0b1111111111111111,16),
                                       zx=_0, nx=_1, zy=_0, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

# x = 17, y = 3
    def test2_0(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_0, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000000,16), 'zr':_1, 'ng':_0})

    def test2_1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test2_minus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_1, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b1111111111111111,16), 'zr':_0, 'ng':_1})

    def test2_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_0),
                         {'out':Signal(0b0000000000010001,16), 'zr':_0, 'ng':_0})

    def test2_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000011,16), 'zr':_0, 'ng':_0})

    def test2_not_x(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b1111111111101110,16), 'zr':_0, 'ng':_1})

    def test2_not_y(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_0, no=_1),
                         {'out':Signal(0b1111111111111100,16), 'zr':_0, 'ng':_1})

    def test2_minusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111101111,16), 'zr':_0, 'ng':_1})

    def test2_minusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b1111111111111101,16), 'zr':_0, 'ng':_1})

    def test2_xplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_1, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000010010,16), 'zr':_0, 'ng':_0})

    def test2_yplus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b0000000000000100,16), 'zr':_0, 'ng':_0})

    def test2_xminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_1, ny=_1, f=_1, no=_0),
                         {'out':Signal(0b0000000000010000,16), 'zr':_0, 'ng':_0})

    def test2_yminus1(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_1, nx=_1, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000000010,16), 'zr':_0, 'ng':_0})

    def test2_xplusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_1, no=_0),
                         {'out':Signal(0b0000000000010100,16), 'zr':_0, 'ng':_0})

    def test2_xminusy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_0, ny=_0, f=_1, no=_1),
                         {'out':Signal(0b0000000000001110,16), 'zr':_0, 'ng':_0})

    def test2_yminusx(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_1, f=_1, no=_1),
                         {'out':Signal(0b1111111111110010,16), 'zr':_0, 'ng':_1})

    def test2_xandy(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_0, zy=_0, ny=_0, f=_0, no=_0),
                         {'out':Signal(0b0000000000000001,16), 'zr':_0, 'ng':_0})

    def test2_xory(self):
        self.assertEqual(self.alu.eval(x=Signal(0b0000000000010001,16), y=Signal(0b0000000000000011,16),
                                       zx=_0, nx=_1, zy=_0, ny=_1, f=_0, no=_1),
                         {'out':Signal(0b0000000000010011,16), 'zr':_0, 'ng':_0})

run_test(TestALU)

....................................
----------------------------------------------------------------------
Ran 36 tests in 24.576s

OK
